In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
% matplotlib inline
from matplotlib import rc
rc('text', usetex=True)
from pylab import rcParams
rcParams['figure.figsize'] = 10,8

In [ ]:
def linearFit(t, m, b): 
    return m*t + b

#defines linear fit function

In [ ]:
data = np.genfromtxt('rawData.csv', delimiter = ',', skip_header=True)
#imports data from text file

In [ ]:
print(data)

In [ ]:
xInFeet = data[ :, 0]
tInNs = data[ :, 1]
dx = [] #total change in distance travelled by the light in meters
dt = [] #change in time

In [ ]:
i = 0
for i in range(len(xInFeet)):
    dx.append(2*0.3048*(xInFeet[i]-xInFeet[10]))
    i+=1
print (dx)
# calculates the total change in distance (twice the mirror's change) and converts it to meters

In [ ]:
j = 0
for j in range(len(tInNs)):
    dt.append(tInNs[j]-tInNs[10])
    j+=1
print (dt)
#calculates the change in time

In [ ]:
dx = np.array(dx)
dt = np.array(dt)
#transforms dx and dt from lists into arrays

Uncertainties: $\Delta x = \pm 0.04m$, $\Delta t = \pm 4 ns$

$c = \frac{dx}{dt}$

$\Delta c = \sqrt{(\frac{\partial c}{\partial x}\Delta x)^2 + (\frac{\partial c}{\partial t}\Delta t)^2}$

$\Delta c = \sqrt{(\frac{1}{t}0.04)^2 + (-xt^{-2}4\times10^{-9})^2}$

In [ ]:
uncertaintyInC = []

In [ ]:
k=0
for k in range(len(dx)-1):
    uncertaintyInC.append(np.sqrt(((0.05/dt[k])**2)+((dx[k]*dt[k])**(-2)*4*10**(-9))**2))
    k+=1
print(uncertaintyInC)

In [ ]:
dc = np.average(uncertaintyInC)
print(dc) #this gives our average uncertainty of the speed of light)

In [ ]:
fitParams, fitCovariances = curve_fit(linearFit, dt, dx)
print(fitParams)
print(fitCovariances)

In [ ]:
sigma = np.sqrt([fitCovariances[0,0], fitCovariances[1,1]])

In [ ]:
plt.plot (dt, dx, 'o')
plt.errorbar(dt, dx, fmt = 'ro', xerr=0.000000004, yerr = .05)
plt.xlabel('Change in Time (s)', fontsize = 14)
plt.ylabel('Change in Distance (M)', fontsize = 14)
plt.title('Speed of Light (M/s)', fontsize = 18)
plt.plot(dt, linearFit(dt, fitParams[0], fitParams[1]), label='$c=3.076 \times 10^8 m/s \pm  $')
plt.legend(bbox_to_anchor=(0.25,0.95))
plt.show